### RDD
- RDD는 스파크의 기본 데이터 구조
- RDD는 단순히 분산되어 존재하는 데이터 요소들의 모임이다.
- 스파크는 RDD에 있는 데이터들을 클러스터에 분배하며 클러스터 위에서 수행하는 연산들을 병렬화한다.
- 분산되어 있는 변경 불가능한 객체 모음

### Spark RDD 사용이유
- mapreduce는 데이터 공유가 느리다.
- MapReduce는 작업을 처리하고 Disk에 저장하고 다음 작업시 Disk에 있는 데이터를 또 읽고 처리한다.
- 메모리에서 데이터 공유는 네트워크나 디스크보다 훨씬 빠르다. Spark RDD는 메모리 내에서 데이터를 처리한다.

### RDD 연산
RDD는 두 가지 타입의 연산을 지원한다.
1. 트랜스포메이션(Transformation)
    - 기존 RDD에서 새로운 RDD를 만든다.
    - Return 값이 RDD이다.
2. 액션(Action) 
    - 출력 및 파일 저장 같은 작업이다.
왜? RDD연산은 두가지 타입의 연산을 지원할까?
=> 스파크는 Lazy Evaluation이라는 방식으로 연산을 처리한다. 액션을 사용하는 시점에 작업이 처리된다. 

예를 들어 내가 파일을 읽고 첫 번째 라인만 조회하려고 한다면, Lazy Evaluation에 의해 첫 번째 라인만 읽는다. 연산 구분이 없으면 모든 파일을 다 읽은 후에 첫번째 라인만 출력하면 스토리지 공간 낭비가 심하다.

또 RDD.persist()를 사용하여 결과를 메모리나 디스크에 저장하여 다음 연산에서 재사용 할 수 있다.

### RDD 생성하기
RDD 생성하는 방법은 두가지 방법이 있다.
1. 외부 데이터 세트 로드
   
   lines = sc.textFile("/path/README.md")
2. 직접 생성
   
   lines = sc.parallelize(["pandas", "i like pandas"])


### 많이 쓰이는 트랜스포메이션과 액션
### 1. 트랜스포메이션
1-1. map
```
nums = sc.parallelize([1,2,3,4])
squared = nums.map(lambda x : x*x).collect()
for num in squared:
    print("%i " %(num))
```